In [62]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler


In [63]:
df = pd.read_csv('./diabetes_prediction_dataset.csv')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [65]:
df.describe().T.round(2)


,count,mean,std,min,25%,50%,75%,max
age,100000.0,41.89,22.52,0.08,24.00,43.00,60.00,80.00
hypertension,100000.0,0.07,0.26,0.00,0.00,0.00,0.00,1.00
heart_disease,100000.0,0.04,0.19,0.00,0.00,0.00,0.00,1.00
bmi,100000.0,27.32,6.64,10.01,23.63,27.32,29.58,95.69
HbA1c_level,100000.0,5.53,1.07,3.50,4.80,5.80,6.20,9.00
blood_glucose_level,100000.0,138.06,40.71,80.00,100.00,140.00,159.00,300.00
diabetes,100000.0,0.08,0.28,0.00,0.00,0.00,0.00,1.00


In [66]:
df['gender'].value_counts()

gender
Female    58552
Male      41430
Other        18
Name: count, dtype: int64

In [67]:
# Mapeamento dos valores
gender_map = {'Male': 0, 'Female': 1, 'Other': 2}

# Aplicando o mapeamento à coluna 'gender'
df['gender'] = df['gender'].map(gender_map)


In [68]:
df['gender'].value_counts()

gender
1    58552
0    41430
2       18
Name: count, dtype: int64

In [69]:
df['smoking_history'].value_counts()

smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64

In [70]:
smoking_map = {
    'No Info': 0,        # Sem informação
    'never': 1,          # Nunca fumou
    'former': 2,         # Ex-fumante
    'not current': 2,    # Não é fumante atualmente (pode ser similar a "former")
    'ever': 3,           # Já fumou em algum momento (pode ser entre "former" e "current")
    'current': 4         # Fumante atual
}

df['smoking_history'] = df['smoking_history'].map(smoking_map)


In [71]:
df['smoking_history'].value_counts()

smoking_history
0    35816
1    35095
2    15799
4     9286
3     4004
Name: count, dtype: int64

In [72]:
df['diabetes'].value_counts()

diabetes
0    91500
1     8500
Name: count, dtype: int64

In [73]:
X = df.drop(columns=['diabetes'], axis=1)

y = df['diabetes']

In [74]:
X.shape, y.shape

((100000, 8), (100000,))

In [75]:
# Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
#                                                 test_size=.2,
#                                                 stratify=y,
#                                                 random_state=42)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                                test_size=.1,
                                                stratify=y,
                                                random_state=42)

In [76]:
Xtrain.shape, Xtest.shape

((90000, 8), (10000, 8))

In [77]:
# # Aplicar SMOTE no conjunto de treino
# smote = SMOTE(random_state=42)
# Xtrain_smote, ytrain_smote = smote.fit_resample(Xtrain, ytrain)

In [78]:
# Verificar o balanceamento após o SMOTE
# print(f"Distribuição após SMOTE: {ytrain_smote.value_counts()}")

In [79]:
lr = LogisticRegression(class_weight='balanced', max_iter=500)

In [80]:
# Treinando o modelo
lr.fit(X=Xtrain, y=ytrain)

LogisticRegression(class_weight='balanced', max_iter=500)

In [81]:
# Acuracia dos dados de treino

lr.score(Xtrain, ytrain)

0.8867

In [82]:
# Acuracia dos dados de teste
lr.score(Xtest, ytest)

0.8869

In [83]:
ypred = lr.predict(Xtest)

In [84]:

# Gerar o relatório de classificação
report = classification_report(ytest, ypred, target_names=['Non-Diabetic', 'Diabetic'])
print(report)

              precision    recall  f1-score   support

Non-Diabetic       0.99      0.89      0.93      9150
    Diabetic       0.42      0.89      0.57       850

    accuracy                           0.89     10000
   macro avg       0.71      0.89      0.75     10000
weighted avg       0.94      0.89      0.90     10000



In [85]:
# Obter as probabilidades de previsão
y_proba = lr.predict_proba(Xtest)[:, 1]

# Ajustar o limiar
fpr, tpr, thresholds = roc_curve(ytest, y_proba)
# Encontrar o limiar que maximiza o recall
best_threshold = thresholds[np.argmax(tpr)]
print(f"Melhor limiar para maximizar o recall: {best_threshold}")

Melhor limiar para maximizar o recall: 0.03170814270375429


In [86]:
new_pred = np.array([1,	36.0, 0, 0,	4, 23.45, 5.0, 155]).reshape(1, -1)
new_pred

array([[  1.  ,  36.  ,   0.  ,   0.  ,   4.  ,  23.45,   5.  , 155.  ]])

In [87]:
result = lr.predict(new_pred)[0]
result_prob = round(lr.predict_proba(new_pred)[:,1][0], 4)

In [88]:
if result == 1:
  print(f'Resultado: {result} | Existe {result_prob*100}% chance de o(a) pasciente ser diabetico.')
else:
  print(f'Resultado: {result} | Existe {result_prob*100}% chance de o(a) pasciente não ser diabetico.')

Resultado: 0 | Existe 2.08% chance de o(a) pasciente não ser diabetico.
